In [1]:
import cv2
import mediapipe as mp
import pyautogui

import os
import time


In [2]:
import numpy as np

def get_angle(a,b,c):
    radians=np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(np.degrees(radians))
    return angle

def get_distance(points):
    if len(points) < 2:
        return 9999 
    (x1, y1), (x2, y2) = points
    return np.hypot(x2 - x1, y2 - y1)


In [3]:
screen_width,screen_height = pyautogui.size()


mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)


In [4]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
       hand_landmarks = processed.multi_hand_landmarks[0]
       return hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    return None

In [5]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y * screen_height)
        pyautogui.moveTo(x, y)


In [6]:
def is_left_click(landmarks_list, thumb_index_dist):
    angle_index = get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8])
    angle_middle = get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12])
    return angle_index < 50 and angle_middle > 90 and thumb_index_dist > 50

def is_right_click(landmarks_list, thumb_index_dist):
    angle_index = get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8])
    angle_middle = get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12])
    return angle_index > 90 and angle_middle < 50 and thumb_index_dist > 50

def double_click(landmarks_list, thumb_index_dist):
    angle_index = get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8])
    angle_middle = get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12])
    return angle_index < 50 and angle_middle < 50 and thumb_index_dist > 50


def screen_shot(landmarks_list, thumb_index_dist):
    if (get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and 
        get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 50 and 
        thumb_index_dist < 50):

       
        folder_path = "D:\\screenshot"
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

       
        file_name = f"screenshot_{int(time.time())}.png"
        save_path = os.path.join(folder_path, file_name)

        # Save screenshot
        pyautogui.screenshot(save_path)
        print(f"✅ Screenshot saved at: {save_path}")
        return True

    return False



In [7]:
def is_scroll(landmarks_list):
 
    index_angle = get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8])
    thumb_angle = get_angle(landmarks_list[1], landmarks_list[2], landmarks_list[4])
    middle_angle = get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12])
    ring_angle = get_angle(landmarks_list[13], landmarks_list[14], landmarks_list[16])
    pinky_angle = get_angle(landmarks_list[17], landmarks_list[18], landmarks_list[20])

    return (
        index_angle > 90 and
        thumb_angle > 90 and
        middle_angle < 50 and
        ring_angle < 50 and
        pinky_angle < 50
    )

prev_scroll_y = None
def perform_scroll(index_tip_y):
    global prev_scroll_y
    if prev_scroll_y is not None:
        dy = prev_scroll_y - index_tip_y
        if abs(dy) > 5:
            pyautogui.scroll(int(dy * 5))  # Adjust sensitivity
    prev_scroll_y = index_tip_y


In [8]:
def detect_gesture(frame, landmarks_list, processed):
    if len(landmarks_list) >= 13:
        index_finger_tip = find_finger_tip(processed)
        thumb_tip = landmarks_list[4]
        index_base = landmarks_list[5]
        thumb_index_dist = get_distance([thumb_tip, index_base])

        
        if is_left_click(landmarks_list, thumb_index_dist):   #left click
            pyautogui.click()
            print("Left Click")

        elif is_right_click(landmarks_list, thumb_index_dist):    #Right click
            pyautogui.click(button='right')
            print("Right Click")

        elif double_click(landmarks_list, thumb_index_dist):    #double click
            pyautogui.doubleClick()
            print("Double Click")

        elif screen_shot(landmarks_list, thumb_index_dist):     # Screen_shot
            pyautogui.screenshot("screenshot.png")
            print("Screenshot Saved")
        elif thumb_index_dist < 40:
                                                   # Move mouse only when thumb touches index base
            move_mouse(index_finger_tip)

        elif is_scroll(landmarks_list):         # Scroll
            perform_scroll(landmarks_list)
        else:
            global prev_scroll_y
            prev_scroll_y = None


In [9]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)

def main():
    cap = cv2.VideoCapture(0)

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            processed = hands.process(frameRGB)

            landmarks_list = []

            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]  
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                for lm in hand_landmarks.landmark: 
                    h, w, _ = frame.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    landmarks_list.append((cx, cy))

            detect_gesture(frame,landmarks_list,processed)

            

            cv2.imshow("frame", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


d:\r_env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Right Click
Left Click
